In [24]:
# %pip install sentence-transformers pinecone-datasets pandas pincone-client

In [25]:
import pandas as pd
dataset = pd.read_csv('E:\Documents\KYH\Data\Quran\sahihinternational.csv')

#remove index column
#dataset = dataset.drop(columns=['Unnamed: 0'])

In [26]:
print(len(dataset))
dataset.head()

6236


,id,surah,verse,text
0,1:1,1,1,"In the name of Allah, the Entirely Merciful, t..."
1,1:2,1,2,"All praise is due to Allah, Lord of the worlds -"
2,1:3,1,3,"The Entirely Merciful, the Especially Merciful,"
3,1:4,1,4,Sovereign of the Day of Recompense.
4,1:5,1,5,It is You we worship and You we ask for help.


In [27]:
df = dataset['text'].copy()

df.head()

0    In the name of Allah, the Entirely Merciful, t...
1     All praise is due to Allah, Lord of the worlds -
2      The Entirely Merciful, the Especially Merciful,
3                  Sovereign of the Day of Recompense.
4        It is You we worship and You we ask for help.
Name: text, dtype: object

In [28]:
df[70:80]

70    Then you turned away after that. And if not fo...
71    And you had already known about those who tran...
72    And We made it a deterrent punishment for thos...
73    And recall when Moses said to his people, "Ind...
74    They said, "Call upon your Lord to make clear ...
75    They said, "Call upon your Lord to show us wha...
76    They said, "Call upon your Lord to make clear ...
77    He said, "He says, 'It is a cow neither traine...
78    And recall when you slew a man and disputed ov...
79    So, We said, "Strike the slain man with part o...
Name: text, dtype: object

In [29]:
df[128+63]

"The month of Ramadhan is that in which was revealed the Qur'an, a guidance for the people and clear proofs of guidance and criterion. So whoever sights the new moon of the month, let him fast it; and whoever is ill or on a journey - then an equal number of other days. Allah intends for you ease and does not intend for you hardship and wants for you to complete the period and to glorify Allah for that to which He has guided you; and perhaps you will be grateful."

In [32]:
df = df.str[:255]

df.reset_index(drop=True, inplace=True)

In [33]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

e:\Documents\KYH\Semantic_Search\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


e:\Documents\KYH\Semantic_Search\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [34]:
input = 'Satan leads mankind astray'

xq = model.encode(input)
xq.shape

(384,)

In [35]:
_id = '0'
metadata = {'text': input}

vectors = [(_id, xq, metadata)]

In [36]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="e6016e2c-2539-4f49-820b-f077fcb07cc1")
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [37]:
index_name = 'semantic-search'

In [38]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 128}},
 'total_vector_count': 128}

In [40]:
from tqdm.auto import tqdm

batch_size = 128

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in df[i:i_end]]
    # create embeddings
    xc = model.encode(df.loc[i:i_end].to_list())
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

    print('batch_upserted', i)

# check number of records in the index
index.describe_index_stats()

  2%|▏         | 1/49 [00:03<03:07,  3.90s/it]

batch_upserted 0


  4%|▍         | 2/49 [00:06<02:20,  3.00s/it]

batch_upserted 128


  6%|▌         | 3/49 [00:08<01:58,  2.58s/it]

...

 98%|█████████▊| 48/49 [01:57<00:02,  2.03s/it]

batch_upserted 6016


100%|██████████| 49/49 [01:59<00:00,  2.43s/it]

batch_upserted 6144


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6144}},
 'total_vector_count': 6144}

In [41]:
query = "Satan leads mankind astray"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': '2811',
              'metadata': {'text': 'O you who have believed, do not follow the '
                                   'footsteps of Satan. And whoever follows '
                                   'the footsteps of Satan - indeed, he '
                                   'enjoins immorality and wrongdoing. And if '
                                   'not for the favor of Allah upon you and '
                                   'His mercy, not one of you would have been '
                                   'pure, ever, '},
              'score': 0.610865414,
              'values': []},
             {'id': '174',
              'metadata': {'text': 'O mankind, eat from whatever is on earth '
                                   'that is lawful and good and do not follow '
                                   'the footsteps of Satan. Indeed, he is to '
                                   'you a clear enemy.'},
              'score': 0.607461035,
              'values': [

In [42]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.61: O you who have believed, do not follow the footsteps of Satan. And whoever follows the footsteps of Satan - indeed, he enjoins immorality and wrongdoing. And if not for the favor of Allah upon you and His mercy, not one of you would have been pure, ever, 
0.61: O mankind, eat from whatever is on earth that is lawful and good and do not follow the footsteps of Satan. Indeed, he is to you a clear enemy.
0.58: My Lord, indeed they have led astray many among the people. So whoever follows me - then he is of me; and whoever disobeys me - indeed, You are yet Forgiving and Merciful.
0.57: Whoever Allah sends astray - there is no guide for him. And He leaves them in their transgression, wandering blindly.
0.57: And never let Satan avert you. Indeed, he is to you a clear enemy.


In [43]:
query = "which metropolis has the highest number of people?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.45: And the people of the city came rejoicing.
0.45: And how many a city which was unjust have We shattered and produced after it another people.
0.42: I swear by this city, Makkah -
0.4: And there is no city but that We will destroy it before the Day of Resurrection or punish it with a severe punishment. That has ever been in the Register inscribed.
0.38: And how many a city have We destroyed that was insolent in its way of living, and those are their dwellings which have not been inhabited after them except briefly. And it is We who were the inheritors.


In [ ]:
# pc.delete_index(index_name)